In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('mel_features_dataset.csv')

In [3]:
X = np.array(df['Mel_Features'].tolist())
y = np.array(df['Ragas'].tolist())

In [10]:
print("Shape of X before reshape:", X.shape)
print("Shape of y before encode:", y.shape)

Shape of X before reshape: (92,)
Shape of y before encode: (92,)


In [4]:
le=LabelEncoder()
ti=le.fit_transform(y)
y_mel=to_categorical(ti)

In [5]:
y

array(['Bhairava', 'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava',
       'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava',
       'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava',
       'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava',
       'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava',
       'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava', 'Bhairava',
       'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali',
       'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali',
       'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali',
       'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali',
       'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali',
       'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali', 'Bhoopali',
       'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yaman',
       'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yaman',
       'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yaman', 'Yam

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=127)

In [11]:
num_classes = len(y[0])
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train_flat = np.array([x.flatten() for x in X_train])

In [12]:
y_train_numerical = le.fit_transform(y_train)
y_val_numerical = le.transform(y_test)
y_train_encoded = to_categorical(y_train_numerical, num_classes=num_classes)
y_val_encoded = to_categorical(y_val_numerical, num_classes=num_classes)

In [13]:
print(y_val_encoded)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]


In [15]:
print("Shape of X_train:", X_train_flat.shape)

Shape of X_train: (73, 1)


In [17]:
model_mel = Sequential()
model_mel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X_train_flat.shape[1], X_train.shape[2], 1)))
model_mel.add(MaxPooling2D((2, 2)))
model_mel.add(Conv2D(64, (3, 3), activation='relu'))
model_mel.add(MaxPooling2D((2, 2)))
model_mel.add(Flatten())
model_mel.add(Dense(128, activation='relu'))
model_mel.add(Dense(64, activation='relu'))
model_mel.add(Dense(num_classes, activation='softmax'))

ValueError: Computed output size would be negative. Received `inputs shape=(None, 1, 1, 1)`, `kernel shape=(3, 3, 1, 32)`, `dilation_rate=[1 1]`.